In [66]:
from lib.agents import *
import numpy as np
from queue import PriorityQueue
from collections import deque

In [43]:
class Wall(Thing):
    pass

class Goal(Thing):
    pass


In [44]:
from random import choice

class OurAgent(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, loc):
        self.location[0] += loc[0]
        self.location[1] += loc[1]
    
    def turn(self, d):
        self.direction = Direction(d)
        
def program(percepts):
    global it
    '''Returns an action based on it's percepts'''
        
    for p in percepts:
        if isinstance(p, Wall):
            print("Cannot move in a wall\nImplement the algorithm again")
            
    it += 1
    return steps[it]

In [45]:
class Maze(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        for thing in self.list_things_at(agent.location):
            if not isinstance(thing, OurAgent):
                things.append(thing)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'right':
            agent.moveforward([1,0])
        elif action == 'left':
            agent.moveforward([-1,0])
        elif action == 'up':
            agent.moveforward([0,-1])
        elif action == 'down':
            agent.moveforward([0,1])
                    
    def is_done(self):
        things = []
        for agent in self.agents:
            for thing in self.list_things_at(agent.location):
                things.append(thing)
        for thing in things:
            if isinstance(thing, Goal):
                return True
        return False


In [140]:
def bfs(maze):
    def get_neighbors(s):
        y_len = len(maze)
        x_len = len(maze[0])
        y, x = s
        neighbors = []
        
        # Check Up
        if y > 0 and maze[y - 1][x] != 1:
            neighbors.append((y - 1, x))
            

        # Check Right
        if x < x_len - 1 and maze[y][x + 1] != 1:
            neighbors.append((y, x + 1))
            

        # Check Down
        if y < y_len - 1 and maze[y + 1][x] != 1:
            neighbors.append((y + 1, x))
            

        # Check Left
        if x > 0 and maze[y][x - 1] != 1:
            neighbors.append((y, x - 1))
            

        return neighbors

    def get_relative_move(s, d):
        dy = d[0] - s[0]
        dx = d[1] - s[1]

        if dy == -1 and dx == 0:
            return "up"
        elif dy == 0 and dx == 1:
            return "right"
        elif dy == 1 and dx == 0:
            return "down"
        elif dy == 0 and dx == -1:
            return "left"
        else:
            return None

    s = tuple(np.argwhere(maze == 3)[0])
    queue = [s]
    visited = {s: None}  # Use to keep track of previous current_location

    while queue:
        current_location = queue.pop(0)
        if maze[current_location] == 2:
            # Backtrack from the goal to the start to get the path
            path = []
            while current_location is not None:
                if visited[current_location] is not None:
                    path.append(get_relative_move(visited[current_location], current_location))
                current_location = visited[current_location]
            return path[::-1]  # Reverse the path
        else:
            neighbors = get_neighbors(current_location)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
                    visited[neighbor] = current_location  # Set previous of path to the neighbor

    return []

In [135]:
def get_neighbors(s,maze):
    y_len = len(maze)
    x_len = len(maze[0])
    y,x=s
    neighbors = []
    cost=[]
    # Check Up
    if y > 0 and maze[y - 1][x] != 1:
        neighbors.append((y - 1, x))
        cost.append(2)
    # Check Right
    if x < x_len - 1 and maze[y][x + 1] != 1:
        neighbors.append((y, x + 1))
        cost.append(4)
    # Check Down
    if y < y_len - 1 and maze[y + 1][x] != 1:
        neighbors.append((y + 1, x))
        cost.append(3)
    # Check Left
    if x > 0 and maze[y][x - 1] != 1:
        neighbors.append((y, x - 1))
        cost.append(1)

    return cost,neighbors


def get_relative_move(s, d):
    dy = d[0] - s[0]
    dx = d[1] - s[1]

    if dy == -1 and dx == 0:
        return "up"
    elif dy == 0 and dx == 1:
        return "right"
    elif dy == 1 and dx == 0:
        return "down"
    elif dy == 0 and dx == -1:
        return "left"
    else:
        return None

In [138]:
def UCS(maze):
    queue = PriorityQueue()
    start = tuple(np.argwhere(maze == 3)[0])
    goal = tuple(np.argwhere(maze == 2)[0])
    queue.put((0, start)) 
    visited = {start: None}
    path = []
    path.append(start)
    while not queue.empty():
        _,current_location = queue.get()
        path.append(current_location)
        if current_location == goal:
            while current_location is not None:
                if visited[current_location] is not None:
                    path.append(get_relative_move(visited[current_location], current_location))
                current_location = visited[current_location]
            return path[::-1]  
        else:
            costs, neighbors = get_neighbors(current_location, maze)
            for neighbor_cost, neighbor in zip(costs, neighbors):
                if neighbor not in visited:
                    queue.put((neighbor_cost, neighbor)) 
                    visited[neighbor] = current_location


In [176]:
def Manhattan_distance(current_location, goal_state, maze):
    _, neighbors = get_neighbors(current_location, maze)
    manhattan_queue = []
    for neighbor in neighbors:
        manhattan_queue.append(abs(goal_state[0] - neighbor[0]) + abs(goal_state[1] - neighbor[1]))
    manhattan_queue.sort()  # Sort the list of Manhattan distances
    return manhattan_queue, neighbors 

In [177]:
def Best_firstSearch(maze):
    queue = PriorityQueue()
    start = tuple(np.argwhere(maze == 3)[0])
    goal = tuple(np.argwhere(maze == 2)[0])
    queue.put((0, start)) 
    visited = {start: None}
    path = []
    path.append(start)
    while not queue.empty():
        _,current_location = queue.get()
        path.append(current_location)
        if current_location == goal:
            while current_location is not None:
                if visited[current_location] is not None:
                    path.append(get_relative_move(visited[current_location], current_location))
                current_location = visited[current_location]
            return path[::-1]  
        else:
            manhattanDistance, neighbors = Manhattan_distance(current_location, goal, maze)
            for distance, neighbor in zip(manhattanDistance, neighbors):
                if neighbor not in visited:
                    queue.put((distance, neighbor))  # Fix the put operation
                    visited[neighbor] = current_location
    return []

Now that our maze_instance is ready for the 2D motion of our energetic dog, lets test it!

In [178]:
maze_layout = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 3, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]


maze_instance = Maze(20, 20, color={'Goal': (0, 128, 0), 'Wall': (165,42,42), 'OurAgent': (255, 255, 0)})

def algo(maze):
    """
    implement this function using the maze from input
    3 in the maze represents the location of the agent
    2 represents the goal
    1 represents the wall
    0 represent the free space in which our agent can walk
    return an array of moves like this e.g. ["down", "right", "up", "left", "left", "down"]
    so that the agent will be at the goal 
    use maze_instance.run(0) to show updates without running logic
    """
    
    
    # return bfs(maze)
    return Best_firstSearch(maze)



# Add walls to the maze_instance
for i in range(len(maze_layout)):
    for j in range(len(maze_layout[0])):
        if maze_layout[i][j] == 1:
            maze_instance.add_thing(Wall(), [i, j])


agent = OurAgent(program)
goal = Goal()
start = [1,1]
end = [18,18]
maze_instance.add_thing(agent, start)
maze_instance.add_thing(goal, end)
maze_instance.exogenous_change()
it = -1
            
maze_instance.delete_thing(agent)
maze_instance.delete_thing(goal)

steps = algo(np.transpose(maze_layout))
# maze_instance.add_thing(agent, start)
# maze_instance.add_thing(goal, end)
# maze_instance.run(len(steps))

TypeError: 'NoneType' object is not iterable